**Notebook baseado em https://www.machinelearningplus.com/nlp/topic-modeling-python-sklearn-examples/**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
rock_musics = 'https://raw.githubusercontent.com/nazareno/palavras-nas-letras/master/letras-ptbr-rock-grande.csv'
data = pd.read_csv(rock_musics)
data.sample(10)

,SName,Lyric,Artist,Songs,Popularity,Genre,Genres
1091,Só Por Uma Noite,Eu procurei em outros corpos encontrar você. E...,Charlie Brown Jr,208,25.7,Rock,Pop/Rock; Rap; Rock; Reggae; Rock Alternativo;...
1803,A Semana Inteira,A semana inteira 1970. Erasmo Carlos - Roberto...,Erasmo Carlos,225,2.3,Rock,Rock; Jovem Guarda; Romântico; MPB; Soul Music...
5310,Eu Vou Me Salvar,"Eu vou me salvar, eu vou me salvar. Eu vou me ...",Rita Lee,297,7.6,Rock,Rock; Pop/Rock; MPB; Rock Alternativo; Jovem G...
7590,Do Mesmo Jeito,Falhei quando tentei dançar a dois. A valsa es...,Skank,139,12.3,Rock,Pop/Rock; Rock; Pop; Rock Alternativo; MPB; Ro...
6406,I Saw You Saying (That You Say That You Saw),Reconheci... a Madonna ali parada no jardim. N...,Ultraje A Rigor,90,2.1,Rock,Rock; Pop/Rock; Rock Alternativo; Punk Rock; M...
6642,Rafaela Eu Amo Sua Mãe,Quando eu te conheci achei que estava apaixona...,Velhas Virgens,137,2.4,Rock,Rock; Blues; Punk Rock; Hard Rock; Rock Altern...
1398,"Senhor, Seu Troco",Agora que você já derrotou. Mais um inimigo. S...,Dead Fish,129,0.7,Rock,Hardcore; Rock; Punk Rock; Regional; Rap; Ska;...
3456,Pensa em Mim,Tudo certo. Gosto mesmo de você. Falo serio. J...,Malta,43,1.4,Rock,Romântico; Rock; Rock Alternativo; Pop/Rock; H...
3153,Quem Sabe,Quem sabe o que é ter e perder alguém?. Quem s...,Los Hermanos,80,5.5,Rock,Rock Alternativo; Rock; MPB; Romântico; Indie;...
5613,Monte Inverno,Monte Inverno. Rosa de Saron. Ouço a voz do ve...,Rosa de Saron,134,8.2,Rock,Gospel/Religioso; Rock; Pop/Rock; Romântico; R...


### **Preprocessamento do texto**

In [ ]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re, nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Adicionando mais stopwords a lista de stopwords
stop_words = set(stopwords.words("portuguese"))
stop_words.update(['que', 'até', 'esse', 
                    'essa', 'pro', 'pra',
                    'oi', 'lá', 'blá', 'bb', 
                    'bbm', 'abm', 'cbm', 
                    'dbm', 'dos', 
                    'ltda', 'editora']), 

# Removendo stopwords das letras de músicas e adicionando caixa baixa
lyrics_rock = []
def preprocessing(lyric,sw=True):
  tokenizer = RegexpTokenizer('\w+')
  tokens = tokenizer.tokenize(lyric)
  if (sw):
    tokens = [word.lower() for word in tokens if word not in stop_words]

  lyrics_rock.append(' '.join(tokens).strip())

data['Lyric'].apply(preprocessing)
data = data.assign(LyricNorm=lyrics_rock)

# Removendo musicas duplicadas
data['SNameNorm'] = data['SName'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
data.drop_duplicates(subset ="SNameNorm", 
                       keep = 'first', inplace = True)

# Filtrando por letras de musicas com mais de 50 palavras
count = data['LyricNorm'].str.split().str.len()
letras = data[count > 50]

print('Letras: {}'.format(letras.shape[0]))
print('\nPor artista: ')
letras['Artist'].value_counts()[0:20]

Letras: 4977

Por artista: 


Rita Lee                   198
Lulu Santos                176
Engenheiros do Hawaii      174
Charlie Brown Jr           174
Erasmo Carlos              174
Capital Inicial            163
Raul Seixas                161
Titãs                      156
Biquini Cavadão            152
Barão Vermelho             134
Os Paralamas do Sucesso    128
Velhas Virgens             128
Fresno                     122
Skank                      120
Jota Quest                 116
Rosa de Saron              107
Detonautas                 105
Pato Fu                    103
Lobão                      101
Nx Zero                    100
Name: Artist, dtype: int64

In [ ]:
# Removendo caracteres espaciais e digitos, especialmente por os dados conterem acordes Ex: G7M, G7, C7m
clean_lyrics = []
for w in range(len(letras['LyricNorm'])):
  lyric = letras['LyricNorm'].iloc[w]

  # remove special characters and digits
  lyric  = re.sub("(\\d|\\W)+|\w*\d\w*"," ",lyric )
  lyric = ' '.join(s for s in lyric.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
  clean_lyrics.append(lyric)

clean_lyrics[0:5]

['enquanto aqui ainda haverá amor com feliz enfrento dor não adianta fugir adianta chorar dia sumir nada irá sobrar sonhar viver todo dia agradecer rezar ser última morrer podem tentar atingir podem mandar onde enquanto aqui vida ainda valor sonhar viver todo dia agradecer rezar ser última morrer uma gota vale tudo quando solução pode resolver fico esperança fico sonhar viver todo dia agradecer rezar ser última morrer sonhar viver todo dia agradecer rezar ser última morrer esperança esperança',
 'deu luz fez sonhar chorou mim fez amar foi mim foi tudo pouco enfim canto sinto falta rezo bem alma queria vivesse vida levou deixou aqui ensinou faz feliz que deus bom lugar guarde mim vez encontro fale quero ouvir onde não vejo sentir foi mim foi tudo pouco enfim canto sinto falta rezo bem alma queria vivesse vida levou deixou aqui ensinou faz feliz que deus bom lugar guarde mim vez encontro',
 'nasce criança acreditar que força esperança continua transformar sonho realidade medo salvação co

### **Iniciando o uso do LDA**


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# COUNT vectorizer (Usamos o CountVectorizer pois precisamos apenas das frequencias das palavras, ou seja usamos o tf e não o tf-idf)
tf_vectorizer = CountVectorizer(
        min_df = 30,
        max_df = 0.5,
        max_features = 10000,
        stop_words = stop_words, 
        ngram_range = (1,2)
  )

#transform
vec_text = tf_vectorizer.fit_transform(clean_lyrics)

#returns a list of words.
words = tf_vectorizer.get_feature_names()

print(vec_text.shape)
print(len(words))
print(words[0:5])

(4977, 1673)
1673
['aberta', 'abismo', 'abra', 'abraça', 'abraçar']


#### Funções auxiliares para visualização dos tópicos

In [ ]:
def print_top_words(model, feature_names, n_top_words):
  for topic_idx, topic in enumerate(model.components_):
    print("\n--\nTopic #{}: ".format(topic_idx + 1))
    message = ", ".join([feature_names[i]
                          for i in topic.argsort()[:-n_top_words - 1:-1]])
    print(message)
  print()

def display_topics(W, H, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("\n--\nTopic #{}: ".format(topic_idx + 1))
        print(", ".join([feature_names[i]
                for i in topic.argsort()[:-no_top_words - 1:-1]]).upper())
        top_d_idx = np.argsort(W[:,topic_idx])[::-1][0:no_top_documents]
        for d in top_d_idx: 
          doc_data = letras[['Artist', 'SName']].iloc[d]
          print('{} - {} : \t{:.2f}'.format(doc_data[1], doc_data[0], W[d, topic_idx]))

#### Usando o GridSearch para encontrar o melhor K para o LDA

In [ ]:
from sklearn.model_selection import GridSearchCV

# Parametros para o GridSearch
search_params = {'n_components': [10, 15, 20], 'learning_decay': [.5, .7, .9]}

lda = LatentDirichletAllocation()
model = GridSearchCV(lda, param_grid=search_params)

model.fit(vec_text)

In [ ]:
# Melhor modelo
best_lda_model = model.best_estimator_

# Melhores parametros
print("Melhores parametros do modelo: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(vec_text))

Melhores parametros do modelo:  {'learning_decay': 0.5, 'n_components': 10}
Best Log Likelihood Score:  -458532.8828223911
Model Perplexity:  752.6474284770103


#### Escolhendo K = 10 e learning decay = 0.5

In [ ]:
lda = LatentDirichletAllocation(n_components=10, 
                                learning_method='online',
                                learning_decay=0.5,
                                random_state=0)

lda.fit(vec_text)
doc_topic_matrix = lda.transform(vec_text)

In [ ]:
print('Matriz documento-tópicos:' + str(doc_topic_matrix.shape))
print('Matriz tópicos-termos:' + str(lda.components_.shape))

Matriz documento-tópicos:(4977, 10)
Matriz tópicos-termos:(10, 1673)


In [ ]:
display_topics(doc_topic_matrix,
               lda.components_, 
               words,
               letras,
               15, 
               10)


--
Topic #1: 
DEUS, MAR, TERRA, CÉU, CHÃO, MUNDO, ROCK, CIDADE, CASA, ONDE, SANGUE, DINHEIRO, FOGO, SOL, CADA
Círculos, Loops E Repetições - Barão Vermelho : 	0.95
Miséria - Titãs : 	0.84
Muito Além - Ira! : 	0.79
Rock do Diabo - Raul Seixas : 	0.76
Rock do Diabo (cover Raul Seixas) - Lobão : 	0.76
Cantai Que o Salvador Chegou - PG : 	0.75
As Loucas - Rita Lee : 	0.74
Glória - Oficina G3 : 	0.72
Comendo Vidro - Barão Vermelho : 	0.70
Saideira (Feat. Samuel Rosa) - Santana : 	0.69

--
Topic #2: 
GENTE, VAI, VAMOS, FICAR, TUDO, VIDA, BEM, TODA, BABY, DIZ, SEMPRE, SER, AGORA, QUALQUER, VERDADE
Conto de Fadas - Barão Vermelho : 	0.82
Ilusão da Perfeição - Aliados : 	0.80
Como Sempre Foi - Aliados : 	0.75
Vamos Detonar - Detonautas : 	0.74
Tudo Está Parado - Jota Quest : 	0.70
No Meio de Tudo, Você - Engenheiros do Hawaii : 	0.70
Paula E Bebeto - Emmerson Nogueira : 	0.68
Hora do Brasil - RPM : 	0.65
Tudo Está Parado (Dj Cuca & Mister Jam - Dont Stop Remix) - Jota Quest : 	0.63
Já Te Falei

### Análise dos tópicos encontrados

1.   Tópico: Palavras aonde buscaram músicas que possuem um teor religioso ou refências a algo a ser adorado
2.   Tópico: Palavras aonde buscaram músicas mais good vibes
3.   Tópico: Não identificado
4.   Tópico: Palavras aonde buscaram músicas que aparentam ter um teor sexual 
5.   Tópico: Palavras aonde buscaram músicas que aparentam ter um grau motivacional
6.   Tópico: Palavras aonde buscaram músicas que falam de amor
7.   Tópico: Não identificado
8.   Tópico: Palavras que aparentam trazer uma sensação de felicidade
9.   Tópico: Palavras que trazem muito a sensação de um lugar ou para aonde ir ou voltar etc.
10.   Tópico: Palavras que trazem uma sensação do tempo, do que há para viver.




### Visualizações

#### Preparando os dados para visualização

In [ ]:
main_topic = []
mt_prob = []
for l in range(len(letras['Artist'])):
  main_topic.append(doc_topic_matrix[l,:].argmax() + 1)
  mt_prob.append(doc_topic_matrix[l,:].max())

letras = letras.assign(main_topic = main_topic, main_topic_prob = mt_prob)

# column names
topicnames = ["Topic {}".format(i + 1) for i in range(doc_topic_matrix.shape[1])]

letra_topico = pd.DataFrame(np.round(doc_topic_matrix, 2), columns=topicnames, index = letras.index)
letra_topico[['Artist', 'SName']] = letras[['Artist','SName']]

ordem = ['Artist', 'SName']
ordem.extend(topicnames)
letra_topico = letra_topico[ordem]

letra_topico.query('Artist == "PG"').sort_values(by = 'Topic 1', ascending = False).head()

,Artist,SName,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10
4507,PG,Cantai Que o Salvador Chegou,0.75,0.01,0.01,0.07,0.01,0.01,0.01,0.07,0.08,0.01
4519,PG,Do Céu Ao Inferno,0.57,0.00,0.00,0.00,0.12,0.17,0.13,0.00,0.00,0.00
4472,PG,Meu Prazer,0.45,0.00,0.00,0.00,0.00,0.50,0.00,0.00,0.00,0.04
4488,PG,A Quem Eu Irei,0.43,0.00,0.10,0.06,0.00,0.15,0.19,0.05,0.00,0.00
4600,PG,Verdadeiro Adorador,0.42,0.00,0.12,0.03,0.00,0.17,0.03,0.04,0.00,0.20


*   No tópico 1 observamos que possuem palavras falavam sobre algo religioso. O PG por sua vez é um cantor cristão e podemos observar que outras músicas dele também tiveram boas probabilidades nesse tópico.
*   Um fato curioso é que são músicas que falam também sobre o amor de Deus, portanto o tópico 6 que são palavras relacionadas a amor, também tiveram probabilidades mais altas que os demais nas músicas do PG



In [ ]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

     |████████████████████████████████| 1.6MB 8.8MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=7fdcb8222eab57c1c91c9e81d45992c861073cbd319c0173a7f8fd6b4ad2bad7
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [ ]:
pyLDAvis.sklearn.prepare(lda, vec_text, tf_vectorizer, sort_topics=False, mds = 'tsne')

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0      32.420952   8.115230       1        1  11.293099
1       0.648955  71.707100       2        1   7.884195
2      71.019684   9.971006       3        1   6.010559
3       3.785981  33.185135       4        1   6.265129
4     -42.666500  -9.222070       5        1  10.051203
5      -3.621689 -43.526516       6        1   9.655844
6      -3.632402  -4.171708       7        1   9.198323
7     -36.584400  35.804413       8        1  11.867256
8      44.391380  50.615383       9        1   7.359545
9      40.235809 -31.591665      10        1  20.414847, topic_info=         Term         Freq        Total Category  logprob  loglift
1640      vou  4654.000000  4654.000000  Default  30.0000  30.0000
1223    quero  3481.000000  3481.000000  Default  29.0000  29.0000
73       amor  3261.000000  3261.000000  Default  28.0000  28.0000
1538      vai  4630.000000  4630.000000  Default  27.0000  27.0000
655     gente  2119.000000  2119.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
960     nunca   553.803602  1660.722404  Topic10  -4.7801   0.4907
617       fim   433.944648  1088.361936  Topic10  -5.0240   0.6694
1516     tudo   612.635131  4078.034392  Topic10  -4.6791  -0.3067
1339      sei   472.903148  2860.349527  Topic10  -4.9380  -0.2109
327   coração   382.483008  1249.816976  Topic10  -5.1502   0.4048

[645 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1         2  0.987117   abismo
2         5  0.993709     abra
3         3  0.979722   abraça
17        2  0.987636   acende
29       10  0.988914  acordei
...     ...       ...      ...
1663      7  0.991132     your
1665      2  0.472512     água
1665      3  0.163050     água
1665      4  0.362703     água
1668      3  0.991683     ódio

[1314 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [ ]:
from sklearn.manifold import TSNE
import altair as alt

letras_embedded = TSNE(n_components=2, verbose=1, perplexity=40, early_exaggeration=20).fit_transform(doc_topic_matrix)

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 4977 samples in 0.005s...
[t-SNE] Computed neighbors for 4977 samples in 0.507s...
[t-SNE] Computed conditional probabilities for sample 1000 / 4977
[t-SNE] Computed conditional probabilities for sample 2000 / 4977
[t-SNE] Computed conditional probabilities for sample 3000 / 4977
[t-SNE] Computed conditional probabilities for sample 4000 / 4977
[t-SNE] Computed conditional probabilities for sample 4977 / 4977
[t-SNE] Mean sigma: 0.097154
[t-SNE] KL divergence after 250 iterations with early exaggeration: 154.410187
[t-SNE] KL divergence after 1000 iterations: 2.048930


In [ ]:
letras = letras.assign(tsne1 = letras_embedded[:,0], tsne2 = letras_embedded[:,1])

alt.Chart(letras.sample(500)).mark_circle(
    opacity = .7,
    size = 30
).encode(
    x = 'tsne1',
    y = 'tsne2', 
    color = 'main_topic:N',
    size = 'main_topic_prob',
    tooltip = ['Artist', 'SName', 'main_topic', 'main_topic_prob']
).interactive()

alt.Chart(...)

Visualizando os tópicos identificados pelo LDA nas músicas do PG

In [ ]:
alt.Chart(letras.query('Artist == "PG"')).mark_circle(
    opacity = .7,
    size = 30
).encode(
    x = 'tsne1',
    y = 'tsne2', 
    color = 'main_topic:N',
    size = 'main_topic_prob',
    tooltip = ['Artist', 'SName', 'main_topic', 'main_topic_prob']
).interactive()

alt.Chart(...)